<a href="https://colab.research.google.com/github/naenumtou/creditrisk/blob/main/factorsCreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Set auto reload
%reload_ext autoreload
%autoreload 2

In [2]:
# Change directory to Google Drive
%cd /content/drive/My Drive/Colab Notebooks/BScore

/content/drive/My Drive/Colab Notebooks/BScore


In [3]:
# Import libraries
import warnings
import pandas as pd
import numpy as np

# Config
pd.set_option('display.max_columns', None) #Max column width
warnings.simplefilter(action = 'ignore', category = pd.errors.PerformanceWarning) #Off warning

In [4]:
# Import data
transaction = pd.read_parquet(
    'Usedcar_transaction.parquet',
    engine = 'pyarrow'
)
print(f'Total rows: {transaction.shape[0]}\nTotal columns: {transaction.shape[1]}')

Total rows: 792456
Total columns: 31


In [5]:
# Sort transaction
transaction = transaction.sort_values(
    by = ['Br_Con', 'Monthkey']
)

# Rename columns for factors creation
cols = [
        'OUTSTANDING_AMOUNT',
        'TOTAL_PAYMENT_MADE',
        'INSTALLMENT_AMOUNT',
        'PARTIAL_PAYMENT_FLAG',
        'DUE_AMOUNT',
        'OVERDUE_AMOUNT',
        'BUCKETS'
]

outCols = [
           'bal',
           'pay',
           'install',
           'pay_types',
           'due',
           'ovd',
           'del'
]

for col, outCol in zip(cols, outCols):
  transaction = transaction.rename(
      columns = {f'{col}': f'{outCol}'}
  )
  print(f'Changed from {col} --> {outCol}')

Changed from OUTSTANDING_AMOUNT --> bal
Changed from TOTAL_PAYMENT_MADE --> pay
Changed from INSTALLMENT_AMOUNT --> install
Changed from PARTIAL_PAYMENT_FLAG --> pay_types
Changed from DUE_AMOUNT --> due
Changed from OVERDUE_AMOUNT --> ovd
Changed from BUCKETS --> del


In [6]:
# Factor creation
# Define month range
monthRanges = [12, 9, 6, 3]

# Define lag range
lags = 12

for backwardFactor in outCols:
  for i in range(1, lags + 1):
    transaction[f'{backwardFactor}{i}'] = transaction.groupby(['Br_Con'])[backwardFactor].shift(i)
    print(f'Factor {transaction.columns[-1]} created')

Factor bal1 created
Factor bal2 created
Factor bal3 created
Factor bal4 created
Factor bal5 created
Factor bal6 created
Factor bal7 created
Factor bal8 created
Factor bal9 created
Factor bal10 created
Factor bal11 created
Factor bal12 created
Factor pay1 created
Factor pay2 created
Factor pay3 created
Factor pay4 created
Factor pay5 created
Factor pay6 created
Factor pay7 created
Factor pay8 created
Factor pay9 created
Factor pay10 created
Factor pay11 created
Factor pay12 created
Factor install1 created
Factor install2 created
Factor install3 created
Factor install4 created
Factor install5 created
Factor install6 created
Factor install7 created
Factor install8 created
Factor install9 created
Factor install10 created
Factor install11 created
Factor install12 created
Factor pay_types1 created
Factor pay_types2 created
Factor pay_types3 created
Factor pay_types4 created
Factor pay_types5 created
Factor pay_types6 created
Factor pay_types7 created
Factor pay_types8 created
Factor pay_type

In [7]:
# Balance related facotors
# Average balance over the last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'avg_bal_{i}'] = transaction.loc[:, 'bal1': f'bal{i}'].sum(axis = 1) / i
  print(f'Factor {transaction.columns[-1]} created')

# Average of balance to financed amount in the past 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'avg_bal_to_fin_{i}'] = transaction[f'avg_bal_{i}'] / transaction['FINANCED_AMT']
  print(f'Factor {transaction.columns[-1]} created')

# Average balance over the last 3 months to average balance over the last 12, 9, 6 months
for i in monthRanges[:-1]: #With out 3 months
  transaction[f'avg_bal_3_to_avg_bal_{i}'] = np.where(
      transaction[f'avg_bal_{i}'] == 0,
      0,
      transaction['avg_bal_3'] / transaction[f'avg_bal_{i}']
  )
  print(f'Factor {transaction.columns[-1]} created')

# Current balance to average balance over the last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'bal_to_avg_bal_{i}'] = np.where(
      transaction[f'avg_bal_{i}'] == 0,
      0,
      transaction['bal'] / transaction[f'avg_bal_{i}']
  )
  print(f'Factor {transaction.columns[-1]} created')

# Maximum balance in past 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'max_bal_{i}'] = transaction.loc[:, 'bal1': f'bal{i}'].max(axis = 1)
  print(f'Factor {transaction.columns[-1]} created')

# Current balance to maximum balance in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'bal_to_max_bal_{i}'] = np.where(
      transaction[f'max_bal_{i}'] == 0,
      0,
      transaction['bal'] / transaction[f'max_bal_{i}']
  )
  print(f'Factor {transaction.columns[-1]} created')

# Average balance over the last 12, 9, 6, 3 months to maximum balance in last 12, 9, 6, 3 months
for i in monthRanges:
  for j in monthRanges:
    if i < j:
      continue
    else:
      transaction[f'avg_bal_{j}_to_max_bal_{i}'] = np.where(
          transaction[f'max_bal_{i}'] == 0,
          0,
          transaction[f'avg_bal_{j}'] / transaction[f'max_bal_{i}']
      )
      print(f'Factor {transaction.columns[-1]} created')

# Current balance to financed amount
transaction['bal_to_fin'] = transaction['bal'] / transaction['FINANCED_AMT']
print(f'Factor {transaction.columns[-1]} created')

# Minimum balance in past 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'min_bal_{i}'] = transaction.loc[:, 'bal1': f'bal{i}'].min(axis = 1)
  print(f'Factor {transaction.columns[-1]} created')

Factor avg_bal_12 created
Factor avg_bal_9 created
Factor avg_bal_6 created
Factor avg_bal_3 created
Factor avg_bal_to_fin_12 created
Factor avg_bal_to_fin_9 created
Factor avg_bal_to_fin_6 created
Factor avg_bal_to_fin_3 created
Factor avg_bal_3_to_avg_bal_12 created
Factor avg_bal_3_to_avg_bal_9 created
Factor avg_bal_3_to_avg_bal_6 created
Factor bal_to_avg_bal_12 created
Factor bal_to_avg_bal_9 created
Factor bal_to_avg_bal_6 created
Factor bal_to_avg_bal_3 created
Factor max_bal_12 created
Factor max_bal_9 created
Factor max_bal_6 created
Factor max_bal_3 created
Factor bal_to_max_bal_12 created
Factor bal_to_max_bal_9 created
Factor bal_to_max_bal_6 created
Factor bal_to_max_bal_3 created
Factor avg_bal_12_to_max_bal_12 created
Factor avg_bal_9_to_max_bal_12 created
Factor avg_bal_6_to_max_bal_12 created
Factor avg_bal_3_to_max_bal_12 created
Factor avg_bal_9_to_max_bal_9 created
Factor avg_bal_6_to_max_bal_9 created
Factor avg_bal_3_to_max_bal_9 created
Factor avg_bal_6_to_max_b

In [8]:
# Payment related facotors
# Average repayment amount in past 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'avg_pay_{i}'] = transaction.loc[:, 'pay1': f'pay{i}'].sum(axis = 1) / i
  print(f'Factor {transaction.columns[-1]} created')

# Average of repayment amount to instalment amount in the past 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'avg_pay_to_install_{i}'] = transaction[f'avg_pay_{i}'] / transaction['install']
  print(f'Factor {transaction.columns[-1]} created')

# Maximum of repayment amount to installment in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'max_pay_to_install_{i}'] = transaction.loc[:, 'pay1': f'pay{i}'].max(axis = 1) / transaction['install']
  print(f'Factor {transaction.columns[-1]} created')

# Minimum of repayment amount to installment in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'min_pay_to_install_{i}'] = transaction.loc[:, 'pay1': f'pay{i}'].min(axis = 1) / transaction['install']
  print(f'Factor {transaction.columns[-1]} created')

# Number of times that payment was made in last 12, 9, 6, 3 months
# 1 = partial payment
# 2 = fully payment
# 3 = over payment
for n, label in enumerate(['partial', 'fully', 'over']):
  for i in monthRanges:
    transaction[f'n_{label}_pay_{i}'] = transaction.loc[:, 'pay_types1': f'pay_types{i}'].eq(n + 1).sum(axis = 1)
    print(f'Factor {transaction.columns[-1]} created')

# Number of full payments to number of partial payments in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'n_fully_to_n_partial_{i}'] = np.where(
      (
          (transaction[f'n_fully_pay_{i}'] == 0) & (transaction[f'n_partial_pay_{i}'] == 0)
      ),
      -1,
      transaction[f'n_fully_pay_{i}'] / transaction[f'n_partial_pay_{i}']
  )
  print(f'Factor {transaction.columns[-1]} created')

# Longest run of consecutive months which contract was fully repayment in the last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'temp{i}'] = 0
  transaction[f'full_pmt_run_{i}'] = 0
  for j in range(1, i + 1):
    transaction[f'temp{i}'] = np.where(
        transaction[f'pay_types{j}'].eq(2),
        transaction[f'temp{i}'] + 1,
        0
    )
    transaction[f'full_pmt_run_{i}'] = transaction[[f'temp{i}', f'full_pmt_run_{i}']].max(axis = 1)
  transaction = transaction.drop(
      [f'temp{i}'],
      axis = 1
  )
  print(f'Factor {transaction.columns[-1]} created')

# Longest run of consecutive months which contract was any repayment (partial or fully) in the last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'temp{i}'] = 0
  transaction[f'any_pmt_run_{i}'] = 0
  for j in range(1, i + 1):
    transaction[f'temp{i}'] = np.where(
        transaction[f'pay_types{j}'].le(2),
        transaction[f'temp{i}'] + 1,
        0
    )
    transaction[f'any_pmt_run_{i}'] = transaction[[f'temp{i}', f'any_pmt_run_{i}']].max(axis = 1)
  transaction.drop(
      [f'temp{i}'],
      axis = 1
  )
  print(f'Factor {transaction.columns[-1]} created')

# Repayment to due amount
transaction['pay_to_due'] = transaction['pay'] / transaction['due']

# Create lags of repayment amount to due amount
for i in range(1, lags + 1):
  transaction[f'pay_to_due{i}'] = transaction.groupby(['Br_Con'])['pay_to_due'].shift(i)
  print(f'Factor {transaction.columns[-1]} created')

# Average of repayment amount to due amount in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'avg_pay_to_due_{i}'] = transaction.loc[:, 'pay_to_due1': f'pay_to_due{i}'].sum(axis = 1) / i
  print(f'Factor {transaction.columns[-1]} created')

# Maximum of repayment amount to due amount in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'max_pay_to_due_{i}'] = transaction.loc[:, 'pay_to_due1': f'pay_to_due{i}'].max(axis = 1)
  print(f'Factor {transaction.columns[-1]} created')
  
# Minimum of repayment amount to due amount in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'min_pay_to_due_{i}'] = transaction.loc[:, 'pay_to_due1': f'pay_to_due{i}'].min(axis = 1)
  print(f'Factor {transaction.columns[-1]} created')

# Drop repayment to due amount and lags
transaction = transaction.drop(
    transaction.loc[:, 'pay_to_due': f'pay_to_due{lags}'],
    axis = 1
)

Factor avg_pay_12 created
Factor avg_pay_9 created
Factor avg_pay_6 created
Factor avg_pay_3 created
Factor avg_pay_to_install_12 created
Factor avg_pay_to_install_9 created
Factor avg_pay_to_install_6 created
Factor avg_pay_to_install_3 created
Factor max_pay_to_install_12 created
Factor max_pay_to_install_9 created
Factor max_pay_to_install_6 created
Factor max_pay_to_install_3 created
Factor min_pay_to_install_12 created
Factor min_pay_to_install_9 created
Factor min_pay_to_install_6 created
Factor min_pay_to_install_3 created
Factor n_partial_pay_12 created
Factor n_partial_pay_9 created
Factor n_partial_pay_6 created
Factor n_partial_pay_3 created
Factor n_fully_pay_12 created
Factor n_fully_pay_9 created
Factor n_fully_pay_6 created
Factor n_fully_pay_3 created
Factor n_over_pay_12 created
Factor n_over_pay_9 created
Factor n_over_pay_6 created
Factor n_over_pay_3 created
Factor n_fully_to_n_partial_12 created
Factor n_fully_to_n_partial_9 created
Factor n_fully_to_n_partial_6 cr

In [9]:
# Due amount related factors
# Average due amount in the past 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'avg_due_{i}'] = transaction.loc[:, 'due1': f'due{i}'].sum(axis = 1) / i
  print(f'Factor {transaction.columns[-1]} created')

# Maximum due amount in the past 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'max_due_{i}'] = transaction.loc[:, 'due1': f'due{i}'].max(axis = 1)
  print(f'Factor {transaction.columns[-1]} created')

# Current month due amount to average due amount over the last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'due_to_avg_due_{i}'] = np.where(
      transaction[f'avg_due_{i}'] == 0,
      0,
      transaction['due'] / transaction[f'avg_due_{i}']
  )
  print(f'Factor {transaction.columns[-1]} created')

# Average due amount over the last 3 months to average due amount over the last 12, 9, 6 months
for i in monthRanges[:-1]: #With out 3 months
  transaction[f'avg_due_3_to_avg_due_{i}'] = np.where(
      transaction[f'avg_due_{i}'] == 0,
      0,
      transaction['avg_due_3'] / transaction[f'avg_due_{i}']
  )
  print(f'Factor {transaction.columns[-1]} created')

# Number of months since maximum due amount has been made in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'n_last_max_due_{i}'] = transaction.loc[:, 'due1': f'due{i}'].eq(
      transaction[f'max_due_{i}'],
      axis = 0
  ).values.argmax(axis = 1) + 1
  print(f'Factor {transaction.columns[-1]} created')

# Due amount in current month to financed amount
transaction['due_to_fin'] = transaction['due'] / transaction['FINANCED_AMT']
print(f'Factor {transaction.columns[-1]} created')

# Due amount in previous 1-6 months to financed amount
for i in range(6, 0, -1):
  transaction[f'due_to_fin_{i}'] = transaction[f'due{i}'] / transaction['FINANCED_AMT']
  print(f'Factor {transaction.columns[-1]} created')

# Average of due amount to financed amount in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'avg_due_to_fin_{i}'] = transaction[f'avg_due_{i}'] / transaction['FINANCED_AMT']
  print(f'Factor {transaction.columns[-1]} created')

# Maximum of due amount to financed amount in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'max_due_to_fin_{i}'] = transaction[f'max_due_{i}'] / transaction['FINANCED_AMT']
  print(f'Factor {transaction.columns[-1]} created')

# Overdue amount in current month to financed amount
transaction['ovd_to_fin'] = transaction['ovd'] / transaction['FINANCED_AMT']
print(f'Factor {transaction.columns[-1]} created')

# Overdue amount in previous 1-6 months to financed amount
for i in range(6, 0, -1):
  transaction[f'ovd_to_fin_{i}'] = transaction[f'ovd{i}'] / transaction['FINANCED_AMT']
  print(f'Factor {transaction.columns[-1]} created')

# Average overdue amount in past 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'avg_ovd_{i}'] = transaction.loc[:, 'ovd1': f'ovd{i}'].sum(axis = 1) / i
  print(f'Factor {transaction.columns[-1]} created')

# Current month overdue amount to average overdue amount over the last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'ovd_to_avg_ovd_{i}'] = np.where(
      transaction[f'avg_ovd_{i}'] == 0,
      0,
      transaction['ovd'] / transaction[f'avg_ovd_{i}']
  )
  print(f'Factor {transaction.columns[-1]} created')

# Average overdue amount over the last 3 months to average overdue amount over the last 12, 9, 6 months
for i in monthRanges[:-1]:
    transaction[f'avg_ovd_3_to_avg_ovd_{i}'] = np.where(
        transaction[f'avg_ovd_{i}'] == 0,
        0,
        transaction['avg_ovd_3'] / transaction[f'avg_ovd_{i}']
    )
    print(f'Factor {transaction.columns[-1]} created')

# Average of overdue amount to financed amount in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'avg_ovd_to_fin_{i}'] = transaction[f'avg_ovd_{i}'] / transaction['FINANCED_AMT']
  print(f'Factor {transaction.columns[-1]} created')

# Maximum overdue amount in past 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'max_ovd_{i}'] = transaction.loc[:, 'ovd1': f'ovd{i}'].max(axis = 1)
  print(f'Factor {transaction.columns[-1]} created')

# Number of months since maximum overdue amount has been made in last 12, 9, 6, 3 
for i in monthRanges:
  transaction[f'n_last_max_ovd_{i}'] = transaction.loc[:, 'ovd1': f'ovd{i}'].eq(
      transaction[f'max_ovd_{i}'],
      axis = 0
  ).values.argmax(axis = 1) + 1
  print(f'Factor {transaction.columns[-1]} created')

# Maximum overdue amount in the past 12, 9, 6, 3 months to financed amount
for i in monthRanges:
  transaction[f'max_ovd_to_fin_{i}'] = transaction[f'max_ovd_{i}'] / transaction['FINANCED_AMT']
  print(f'Factor {transaction.columns[-1]} created')

Factor avg_due_12 created
Factor avg_due_9 created
Factor avg_due_6 created
Factor avg_due_3 created
Factor max_due_12 created
Factor max_due_9 created
Factor max_due_6 created
Factor max_due_3 created
Factor due_to_avg_due_12 created
Factor due_to_avg_due_9 created
Factor due_to_avg_due_6 created
Factor due_to_avg_due_3 created
Factor avg_due_3_to_avg_due_12 created
Factor avg_due_3_to_avg_due_9 created
Factor avg_due_3_to_avg_due_6 created
Factor n_last_max_due_12 created
Factor n_last_max_due_9 created
Factor n_last_max_due_6 created
Factor n_last_max_due_3 created
Factor due_to_fin created
Factor due_to_fin_6 created
Factor due_to_fin_5 created
Factor due_to_fin_4 created
Factor due_to_fin_3 created
Factor due_to_fin_2 created
Factor due_to_fin_1 created
Factor avg_due_to_fin_12 created
Factor avg_due_to_fin_9 created
Factor avg_due_to_fin_6 created
Factor avg_due_to_fin_3 created
Factor max_due_to_fin_12 created
Factor max_due_to_fin_9 created
Factor max_due_to_fin_6 created
Facto

In [10]:
# Delinquency related factors
# Maximum delinquency in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'max_del_{i}'] = transaction.loc[:, 'del1': f'del{i}'].max(axis = 1)
  print(f'Factor {transaction.columns[-1]} created')

# A contract has delinquency more than 0 day (del >= 1) in last 12, 9, 6, 3 months
# A contract has delinquency more than 30 days (del >= 2) in last 12, 9, 6, 3 months
# A contract has delinquency more than 60 days (del >= 3) in last 12, 9, 6, 3 months
# A contract has delinquency more than 90 days (del >= 4) in last 12, 9, 6, 3 months
for i in monthRanges:
  for j, label in enumerate(['x', '30', '60', '90']):
    transaction[f'ever_{label}_dpd_{i}'] = np.where(
        transaction.loc[:, 'del1': f'del{i}'].ge(j + 1).any(axis = 1),
        1,
        0
    )
    print(f'Factor {transaction.columns[-1]} created')

# Number of times that a contract has delinquency > 0 day (del >= 1) in last 12, 9, 6, 3 months
# Number of times that a contract has delinquency > 30 day (del >= 2) in last 12, 9, 6, 3 months
# Number of times that a contract has delinquency > 60 day (del >= 3) in last 12, 9, 6, 3 months
# Number of times that a contract has delinquency > 90 day (del >= 4) in last 12, 9, 6, 3 months
for i in monthRanges:
  for j, label in enumerate(['x', '30', '60', '90']):
    transaction[f'n_{label}_dpd_{i}'] = transaction.loc[:, 'del1': f'del{i}'].ge(j + 1).sum(axis = 1)
    print(f'Factor {transaction.columns[-1]} created')

# Number of months since last delinquency status as > 0, 30, 60, 90 days
for i, label in enumerate(['x', '30', '60', '90']):
  condition = transaction.loc[:, 'del1': 'del12'].values == i + 1
  transaction[f'n_last_month_{label}'] = np.where(
      np.any(
          condition,
          axis = 1
      ),
      condition.argmax(axis = 1) + 1,
      0
  )

# Longest run of consecutive months which delinquency status as > 0 days in the last 12, 9, 6, 3 months
# Longest run of consecutive months which delinquency status as > 30 days in the last 12, 9, 6, 3 months
# Longest run of consecutive months which delinquency status as > 60 days in the last 12, 9, 6, 3 months
# Longest run of consecutive months which delinquency status as > 90 days in the last 12, 9, 6, 3 months
for i in monthRanges:
  for j, label in enumerate(['x', '30', '60', '90']):
    transaction[f'temp{i}_{j + 1}'] = 0
    transaction[f'delq_{label}_run_{i}'] = 0
    for n in range(1, i + 1):
      transaction[f'temp{i}_{j + 1}'] = np.where(
          transaction[f'del{n}'].ge(j + 1),
          transaction[f'temp{i}_{j + 1}'] + 1,
          0
      )
      transaction[f'delq_{label}_run_{i}'] = transaction[[f'temp{i}_{j + 1}', f'delq_{label}_run_{i}']].max(axis = 1)
    transaction = transaction.drop(
        [f'temp{i}_{j + 1}'],
        axis = 1
    )
    print(f'Factor {transaction.columns[-1]} created')

Factor max_del_12 created
Factor max_del_9 created
Factor max_del_6 created
Factor max_del_3 created
Factor ever_x_dpd_12 created
Factor ever_30_dpd_12 created
Factor ever_60_dpd_12 created
Factor ever_90_dpd_12 created
Factor ever_x_dpd_9 created
Factor ever_30_dpd_9 created
Factor ever_60_dpd_9 created
Factor ever_90_dpd_9 created
Factor ever_x_dpd_6 created
Factor ever_30_dpd_6 created
Factor ever_60_dpd_6 created
Factor ever_90_dpd_6 created
Factor ever_x_dpd_3 created
Factor ever_30_dpd_3 created
Factor ever_60_dpd_3 created
Factor ever_90_dpd_3 created
Factor n_x_dpd_12 created
Factor n_30_dpd_12 created
Factor n_60_dpd_12 created
Factor n_90_dpd_12 created
Factor n_x_dpd_9 created
Factor n_30_dpd_9 created
Factor n_60_dpd_9 created
Factor n_90_dpd_9 created
Factor n_x_dpd_6 created
Factor n_30_dpd_6 created
Factor n_60_dpd_6 created
Factor n_90_dpd_6 created
Factor n_x_dpd_3 created
Factor n_30_dpd_3 created
Factor n_60_dpd_3 created
Factor n_90_dpd_3 created
Factor delq_x_run_1

In [11]:
# Export
transaction.to_parquet(
    'behaviourFactors.parquet',
    engine = 'pyarrow'
)